In [4]:
import pandas as pd
import numpy as np
from typing import Union, Optional

def clean_population_demo(
    data: Union[str, pd.DataFrame],
    output_path: Optional[str] = None
) -> pd.DataFrame:
    """
    Clean and standardize UK population deprivation and demographic data.
    Optionally save the result to a CSV file.

    Parameters
    ----------
    data : str or pd.DataFrame
        Path to input CSV file (e.g., 'population_demo.csv') or a pandas DataFrame.
    output_path : str, optional
        If provided, the cleaned DataFrame will be saved to this CSV path.

    Returns
    -------
    pd.DataFrame
        Cleaned DataFrame with unified IMD columns and selected demographics.
    """
    # Load data if path is provided
    if isinstance(data, str):
        df = pd.read_csv(data)
    elif isinstance(data, pd.DataFrame):
        df = data.copy()
    else:
        raise TypeError("Input 'data' must be a file path (str) or a pandas DataFrame.")

    # Column renaming map
    column_mapping = {
        'eid': 'participant_id',
        # England IMD
        'p26410': 'england_imd', 'p26411': 'england_income', 'p26412': 'england_employment',
        'p26413': 'england_health', 'p26414': 'england_education', 'p26415': 'england_crime',
        'p26416': 'england_housing', 'p26417': 'england_environment',
        # Scotland IMD
        'p26418': 'scotland_imd', 'p26419': 'scotland_income', 'p26420': 'scotland_employment',
        'p26421': 'scotland_health', 'p26422': 'scotland_education', 'p26423': 'scotland_access',
        'p26424': 'scotland_crime', 'p26425': 'scotland_housing',
        # Wales IMD
        'p26426': 'wales_imd', 'p26427': 'wales_income', 'p26428': 'wales_employment',
        'p26429': 'wales_health', 'p26430': 'wales_education', 'p26431': 'wales_access',
        'p26432': 'wales_housing', 'p26433': 'wales_safety', 'p26434': 'wales_environment',
        # Demographics & contact
        'p21022': 'age_recruit', 'p52': 'birth_month', 'p34': 'birth_year', 'p31': 'sex',
        'p22189': 'townsend_index', 'p190': 'height_cm', 'p191': 'weight_kg',
        'p20005': 'email_access', 'p20143': 'last_contact_date',
        'p20144': 'last_contact_mode', 'p20145': 'contact_count'
    }

    df = df.rename(columns=column_mapping)

    # Define IMD domain groups
    domain_groups = {
        'overall': ['england_imd', 'scotland_imd', 'wales_imd'],
        'income': ['england_income', 'scotland_income', 'wales_income'],
        'employment': ['england_employment', 'scotland_employment', 'wales_employment'],
        'health': ['england_health', 'scotland_health', 'wales_health'],
        'education': ['england_education', 'scotland_education', 'wales_education'],
        'housing': ['england_housing', 'scotland_housing', 'wales_housing'],
        'crime': ['england_crime', 'scotland_crime', 'wales_safety'],
        'access': ['england_environment', 'scotland_access', 'wales_access']
    }

    # Create unified IMD columns
    for domain, cols in domain_groups.items():
        df[f'imd_{domain}'] = df[cols].bfill(axis=1).iloc[:, 0]

    # Drop all original IMD columns (including unused 'wales_environment')
    imd_cols_to_drop = [col for cols in domain_groups.values() for col in cols] + ['wales_environment']
    df = df.drop(columns=imd_cols_to_drop, errors='ignore')

    # Final column selection
    final_columns = [
        'participant_id', 'age_recruit', 'sex', 'contact_count',
        'imd_overall', 'imd_income', 'imd_employment', 'imd_health',
        'imd_education', 'imd_housing', 'imd_crime', 'imd_access',
        'birth_month', 'birth_year'
    ]
    available_cols = [col for col in final_columns if col in df.columns]
    df = df[available_cols]

    # Save if output path is given
    if output_path is not None:
        df.to_csv(output_path, index=False)

    return df


# ───────────────────────────────
# ✅ Usage (run this part)
# ───────────────────────────────

# Clean and save in one go (assumes 'population_demo.csv' is in the current working directory)
clean_df = clean_population_demo(
    data='population_demo.csv',
    output_path='population_demo_clean.csv'
)

# View result
clean_df.head(5)

,participant_id,age_recruit,sex,contact_count,imd_overall,imd_income,imd_employment,imd_health,imd_education,imd_housing,imd_crime,imd_access,birth_month,birth_year
0,5574091,65,Female,2,12.75,0.06,0.05,-0.17,5.70,0.48,29.37,23.50,September,1942
1,2695459,52,Male,33,13.00,0.09,0.07,-0.54,15.17,0.27,24.84,4.37,August,1956
2,1909943,63,Male,1,24.80,34.20,41.00,30.10,16.30,25.30,68.50,5.10,April,1944
3,1832174,48,Male,50,10.19,0.04,0.06,0.26,10.47,-0.68,29.75,1.49,August,1961
4,3028420,63,Female,5,22.75,0.13,0.14,0.41,24.31,-0.20,23.63,16.28,December,1946
